## Setup


[Setup Python Virtual Environment in VSCode](/venv_setup.md)


**Security Note:** Using fixed package versions ensures stability and reproducibility, but may expose you to security vulnerabilities if not regularly updated. Balance stability and security by periodically reviewing and updating packages.

In [ ]:
# Install required packages in a virtual environment (Currently only tested with Python 3.12.4)
%pip install --upgrade pip
%pip install pandas==2.2.2
%pip install matplotlib==3.9.2
%pip install requests==2.32.3

# Download JLCPCB Parts CSV File 

In [45]:
import requests
import os


def download_file(url, filename):
    try:
        # Check if the file already exists
        if os.path.exists(filename):
            # Delete the existing file
            os.remove(filename)
            print(f"Deleted existing file: {filename}")

        response = requests.get(f"{url}/{filename}", stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        with open(filename, "wb") as f:
            for chunk in response.iter_content(None):
                f.write(chunk)
        print(f"Downloaded {url}/{filename} to {filename}")
    except requests.RequestException as e:
        print(f"Download {url} failed: {e}")


# URL
url = "https://cdfer.github.io/jlcpcb-parts-database"

# Local file path to save the database file
local_file = "jlcpcb-components-basic-preferred.csv"

# Download the file
download_file(url, local_file)

Deleted existing file: jlcpcb-components-basic-preferred.csv
Downloaded https://cdfer.github.io/jlcpcb-parts-database/jlcpcb-components-basic-preferred.csv to jlcpcb-components-basic-preferred.csv


In [48]:
import json
import re
import pandas as pd

def extract_value(description, mode, lcsc_id):
    if mode == "Resistors":
        pattern = r"(\d+(?:[a-zA-Z]*)?(?:Ω|ohm|Ohm|OHM))"  # matches numbers followed by Ω, ohm, Ohm, or OHM, or any letter then by Ω, ohm, Ohm, or OHM
    elif mode == "Capacitors":
        pattern = r"(\d+(?:[a-zA-Z]*)?(?:F|f|Farad|farad|pF|pf|nF|nf|uF|uf))"  # matches numbers followed by F, f, Farad, farad, pF, pf, nF, nf, uF, uf
    else:
        raise ValueError("extract_value(): Invalid mode")

    match = re.search(pattern, description, re.IGNORECASE)
    if match:
        return match.group(0)
    else:
        print(f"no value extracted for https://jlcpcb.com/partdetail/C{lcsc_id}  ({description})")
        return None


def get_plan_type(df, index):
    if df.loc[index, "basic"] > 0:
        return "Basic Component"
    elif df.loc[index, "preferred"] > 0:
        return "Preferred Component"
    else:
        print("extended component found")
        return "Extended Component"


def generate_header(name):
    symbol = f'\t(symbol "{name}"'
    symbol += "\n\t\t(pin_numbers hide)"
    symbol += "\n\t\t(pin_names\n\t\t\t(offset 0)\n\t\t)"
    symbol += "\n\t\t(exclude_from_sim no)"
    symbol += "\n\t\t(in_bom yes)"
    symbol += "\n\t\t(on_board yes)"
    return symbol


def generate_property(key, value, at, size=1.27, hide="yes", autoplace=True):
    if autoplace == False:
        autoplace_str = "(do_not_autoplace)"
    else:
        autoplace_str = ""
    return f'\n\t\t(property "{key}" "{value}"\n\t\t\t(at {at}){autoplace_str}\n\t\t\t(effects\n\t\t\t\t(font\n\t\t\t\t\t(size {size} {size})\n\t\t\t\t)\n\t\t\t\t(hide {hide})\n\t\t\t)\n\t\t)'


def generate_rectangle(
    start,
    end,
    name,
    index,
    stroke={"width": 0.254, "type": "default"},
    fill={"type": "none"},
):
    return f'\n\t\t(symbol "{name}_{index}_1"\n\t\t\t(rectangle\n\t\t\t\t(start {start})\n\t\t\t\t(end {end})\n\t\t\t\t(stroke\n\t\t\t\t\t(width {stroke["width"]})\n\t\t\t\t\t(type {stroke["type"]})\n\t\t\t\t)\n\t\t\t\t(fill\n\t\t\t\t\t(type {fill["type"]})\n\t\t\t\t)\n\t\t\t)\n\t\t)'


def generate_polyline(
    points,
    name,
    index,
    stroke={"width": 0.254, "type": "default"},
    fill={"type": "none"},
):
    polyline_str = f'\n\t\t(symbol "{name}_{index}_1"\n\t\t\t(polyline\n\t\t\t\t(pts\n'
    for point in points:
        polyline_str += f"\t\t\t\t\t(xy {point})\n"
    polyline_str += f'\t\t\t\t)\n\t\t\t\t(stroke\n\t\t\t\t\t(width {stroke["width"]})\n\t\t\t\t\t(type {stroke["type"]})\n\t\t\t\t)\n\t\t\t\t(fill\n\t\t\t\t\t(type {fill["type"]})\n\t\t\t\t)\n\t\t\t)\n\t\t)'
    return polyline_str


def generate_pins(pin_type, name, index, pin_length = "1.27", pinA=1, pinB=2):
    symbol = f'\n\t\t(symbol "{name}_{index}_1"\n\t\t\t(pin {pin_type}\n\t\t\t\t(at 0 3.81 270)\n\t\t\t\t(length {pin_length})\n\t\t\t\t(name "~"\n\t\t\t\t\t(effects\n\t\t\t\t\t\t(font\n\t\t\t\t\t\t\t(size 1.27 1.27)\n\t\t\t\t\t\t)\n\t\t\t\t\t)\n\t\t\t\t)\n\t\t\t\t(number "{pinA}"\n\t\t\t\t\t(effects\n\t\t\t\t\t\t(font\n\t\t\t\t\t\t\t(size 1.27 1.27)\n\t\t\t\t\t\t)\n\t\t\t\t\t)\n\t\t\t\t)\n\t\t\t)'
    symbol += f'\n\t\t\t(pin {pin_type}\n\t\t\t\t(at 0 -3.81 90)\n\t\t\t\t(length {pin_length})\n\t\t\t\t(name "~"\n\t\t\t\t\t(effects\n\t\t\t\t\t\t(font\n\t\t\t\t\t\t\t(size 1.27 1.27)\n\t\t\t\t\t\t)\n\t\t\t\t\t)\n\t\t\t\t)\n\t\t\t\t(number "{pinB}"\n\t\t\t\t\t(effects\n\t\t\t\t\t\t(font\n\t\t\t\t\t\t\t(size 1.27 1.27)\n\t\t\t\t\t\t)\n\t\t\t\t\t)\n\t\t\t\t)\n\t\t\t)'
    symbol += "\n\t\t)"
    return symbol


def generate_kicad_symbol(
    mode,
    lcsc,
    datasheet,
    description,
    footprint,
    value,
    keywords,
    price,
    component_class,
    stock,
    category,
    manufacturerPartID,
    attributes,
    units,
):
    if mode == "Resistors":
        ref_designator = "R"
        ref_position = "2.032 0 0"
        value_position = "0 0 90"
        value_autoplace = False
    elif mode == "Capacitors":
        ref_designator = "C"
        ref_position = "2.032 0 0"
        value_position = "3.556 -1.524 0"
        value_autoplace = True
    else:
        ref_designator = "NA"
        ref_position = "0 0 0"
        value_position = "0 0 0"
        value_autoplace = True
    name = f"{footprint},{value}"
    lcsc = f"C{lcsc}" 
    if f"{ref_designator}_{footprint}" in footprints_lookup:
        footprint = f"JLCPCB-Kicad-Footprints:{ref_designator}_{footprint}"
    else:
        print(
            f"footprint missing: {ref_designator}_{footprint} https://jlcpcb.com/partdetail/{lcsc} "
        )

    symbol = generate_header(name)

    symbol += generate_property("Reference", ref_designator, ref_position, hide="no")
    symbol += generate_property("Value", value, value_position, size=0.8, hide="no", autoplace=value_autoplace)
    symbol += generate_property("Footprint", footprint, "-1.778 0 90")
    symbol += generate_property("Datasheet", datasheet, "0 0 0")
    symbol += generate_property("Description", description, "0 0 0")
    symbol += generate_property("LCSC", lcsc, "0 0 0")
    symbol += generate_property("Price", price, "0 0 0")
    symbol += generate_property("Class", component_class, "0 0 0")
    symbol += generate_property("Stock", stock, "0 0 0")
    symbol += generate_property("Category", category, "0 0 0")
    symbol += generate_property("Manufacturer", manufacturerPartID, "0 0 0")

    if type(attributes) == dict:
        for key, value in attributes.items():
            symbol += generate_property(f"{key}", f"{value}", "0 0 0")

    symbol += generate_property("ki_keywords", keywords, "0 0 0")
    symbol += generate_property("ki_fp_filters", f"{ref_designator}_*", "0 0 0")

    if mode == "Resistors":
        symbol += generate_rectangle("-1.016 2.54", "1.016 -2.54", name=name, index=0)
        for i in range(1, units + 1):
            symbol += generate_pins("passive line", name, i, "1.27", i, (units * 2) - (i - 1))

    elif mode == "Capacitors":
        symbol += generate_polyline(["-1.27 0.635", "1.27 0.635"], name=name, index=0)
        symbol += generate_polyline(["-1.27 -0.635", "1.27 -0.635"], name=name, index=0)
        polarized_footprints = [
            "C_Plugin,D5xL11mm",
            "C_Plugin,D10xL12.5mm",
        ]
        if any(s in footprint for s in polarized_footprints):
            symbol += generate_polyline(["-1.27 1.27", "-0.635 1.27"], name=name, index=0, stroke={ "width": 0.127,"type": "default" })
            symbol += generate_polyline(["-0.9525 1.5875", "-0.9525 0.9525"], name=name, index=0, stroke={ "width": 0.127,"type": "default" })
        for i in range(1, units + 1):
            symbol += generate_pins("passive line", name, i, "3.175", i, (units * 2) - (i - 1))

    symbol += "\n\t)"
    return symbol


def generate_kicad_symbol_lib(symbols):
    for lib_name, symbol_list in symbols.items():
        lib_content = "(kicad_symbol_lib\n"
        lib_content += "\t(version 20231120)\n"
        lib_content += '\t(generator "CDFER")\n'
        lib_content += '\t(generator_version "8.0")\n'
        for symbol in symbol_list:
            lib_content += symbol + "\n"
        lib_content += ")\n"

        lib_content = lib_content.replace("℃", "°C")

        with open(f"JLCPCB-Kicad-Symbols/JLCPCB-{lib_name}.kicad_sym", "w") as f:
            f.write(lib_content)

df = pd.read_csv(local_file)

footprints_dir = "JLCPCB-Kicad-Footprints"
footprints_lookup = {os.path.splitext(file)[0] for file in os.listdir(footprints_dir)}

symbols = {"Resistors": [], "Capacitors": []}
smt_joint_cost = 0.0017
hand_solder_joint_cost = 0.0173

for index in range(1, len(df)):
    # lcsc,category_id,category,subcategory,mfr,package,joints,manufacturer,basic,preferred,description,datasheet,stock,last_on_stock,price,extra
    lcsc = df.loc[index, "lcsc"]
    category = f'{df.loc[index,"category"]},{df.loc[index,"subcategory"]}'
    manufacturerPartID = f'{df.loc[index,"manufacturer"]} - {df.loc[index,"mfr"]}'
    footprint = df.loc[index, "package"]
    if "x4" in footprint:
        units = 4
    else:
        units = 1
    description = df.loc[index, "description"]
    joints = df.loc[index, "joints"]

    if "Plugin" in footprint:
        joint_cost = hand_solder_joint_cost
    else:
        joint_cost = smt_joint_cost

    price_json = json.loads(df.loc[index, "price"])
    price = float(price_json[0]["price"] + (joints * joint_cost))
    price = round(price, 3)
    price = f"{price}USD"

    component_class = get_plan_type(df, index)
    stock = df.loc[index, "stock"]

    keywords = ""

    value = None

    try:
        extra_json = json.loads(df.loc[index, "extra"])
        datasheet = extra_json["datasheet"]["pdf"]
        attributes = extra_json["attributes"]
    except:
        datasheet = df.loc[index, "datasheet"]
        attributes = []

    if df.loc[index, "category"] == "Resistors":
        value = extract_value(description, "Resistors", lcsc)
        lib_name = "Resistors"
    elif df.loc[index, "category"] == "Capacitors":
        value = extract_value(description, "Capacitors", lcsc)
        lib_name = "Capacitors"

    if value != None:
        df.drop(index = index, inplace= True)
        symbols[lib_name].append(
            generate_kicad_symbol(
                lib_name,
                lcsc,
                datasheet,
                description,
                footprint,
                value,
                keywords,
                price,
                component_class,
                stock,
                category,
                manufacturerPartID,
                attributes,
                units,
            )
        )
    # else:
    #     print(f"error finding value for C{lcsc} >{description}<")

df.to_csv("jlcpcb-leftover.csv", index=False)

generate_kicad_symbol_lib(symbols)

footprint missing: C_Plugin,D10xL12mm https://jlcpcb.com/partdetail/C2835711 
footprint missing: C_Plugin,D10xL12mm https://jlcpcb.com/partdetail/C2920654 
footprint missing: C_Plugin,D10xL14mm https://jlcpcb.com/partdetail/C2895080 
footprint missing: C_Plugin,D10xL20mm https://jlcpcb.com/partdetail/C2937527 
footprint missing: C_Plugin,D12.5xL16mm https://jlcpcb.com/partdetail/C2874047 
footprint missing: C_Plugin,D13xL21mm https://jlcpcb.com/partdetail/C2841501 
footprint missing: C_Plugin,D18xL20mm https://jlcpcb.com/partdetail/C2972651 
footprint missing: C_Plugin,D18xL30mm https://jlcpcb.com/partdetail/C2841137 
footprint missing: C_Plugin,D18xL36mm https://jlcpcb.com/partdetail/C2841497 
footprint missing: C_Plugin,D8xL12mm https://jlcpcb.com/partdetail/C2891869 
footprint missing: C_Plugin,D8xL16mm https://jlcpcb.com/partdetail/C2923844 
footprint missing: C_Plugin https://jlcpcb.com/partdetail/C360353 
footprint missing: C_Plugin,P=5.08mm https://jlcpcb.com/partdetail/C2761725